# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Objetivo" data-toc-modified-id="Objetivo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objetivo</a></div><div class="lev1 toc-item"><a href="#Metodología" data-toc-modified-id="Metodología-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Metodología</a></div><div class="lev1 toc-item"><a href="#Definición-de-funciones-(estas-luego-estarán-en-un-script)" data-toc-modified-id="Definición-de-funciones-(estas-luego-estarán-en-un-script)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Definición de funciones (estas luego estarán en un script)</a></div><div class="lev1 toc-item"><a href="#Prueba-de-método:-lo-aplicamos-a-una-especie" data-toc-modified-id="Prueba-de-método:-lo-aplicamos-a-una-especie-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Prueba de método: lo aplicamos a una especie</a></div><div class="lev2 toc-item"><a href="#Get-cells-for-the-selected-species" data-toc-modified-id="Get-cells-for-the-selected-species-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Get cells for the selected species</a></div><div class="lev2 toc-item"><a href="#Get-cell-scores" data-toc-modified-id="Get-cell-scores-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Get cell scores</a></div><div class="lev2 toc-item"><a href="#De-los-resultados-seleccionamos-solo-las-celdas-que-ocupa-la-especie" data-toc-modified-id="De-los-resultados-seleccionamos-solo-las-celdas-que-ocupa-la-especie-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>De los resultados seleccionamos solo las celdas que ocupa la especie</a></div><div class="lev2 toc-item"><a href="#Prueba-considerando-sólo-celdas-ocupadas-por-al-menos-una-especie-del-grupo-de-covariables" data-toc-modified-id="Prueba-considerando-sólo-celdas-ocupadas-por-al-menos-una-especie-del-grupo-de-covariables-44"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Prueba considerando sólo celdas ocupadas por al menos una especie del grupo de covariables</a></div><div class="lev1 toc-item"><a href="#Aplicar-el-método-para-todos-los-mamíferos" data-toc-modified-id="Aplicar-el-método-para-todos-los-mamíferos-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Aplicar el método para todos los mamíferos</a></div><div class="lev2 toc-item"><a href="#Get-the-most-suspicious-observations" data-toc-modified-id="Get-the-most-suspicious-observations-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Get the most suspicious observations</a></div><div class="lev2 toc-item"><a href="#Get-the-most-problematic-species" data-toc-modified-id="Get-the-most-problematic-species-52"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Get the most problematic species</a></div><div class="lev1 toc-item"><a href="#Por-hacer" data-toc-modified-id="Por-hacer-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Por hacer</a></div><div class="lev1 toc-item"><a href="#Notas" data-toc-modified-id="Notas-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Notas</a></div>

# Objetivo

Evaluar si un registro de ejemplar tiene coordenadas raras (o sea que se alejan de lo que se observa para los otros registros de la misma especie). Si esto funciona podríamos acelerar el proceso de validación geográfica de datos. 

# Metodología

* Construir un modelo de distribución potencial para cada especie y marcar registros que caen en celdas con score negativo
* Para calcular el score utilizaremos todo la información disponible (todas las especies del SNIB que tengan más de 10 registros, y capas climáticas) 
* El modelo lo calculamos usando sólo las celdas que tiene al menos un registro en el SNIB (podríamos ser más estrictos y pedir al menos k > 1 registros) 

# Definición de funciones (estas luego estarán en un script)
En esta sección definimos algunas funciones que necesitamos a lo largo del notebook

In [ ]:
# Load libraries
from __future__ import print_function
from __future__ import division

import requests
import json
import pandas as pd

import folium
import folium.plugins
import os

import math

Las constantes API_NICHE_URL y API_TAXA_URL definen el URL de los endpoints que vamos a usar. El pirmero tiene todas los servicios para obtener los scores y las celdas de las especies, y el segundo tiene servicios para obtener información taxonómica.

In [ ]:
# URL of SPECIES 1.0 API
# API_NICHE_URL = u'http://species.conabio.gob.mx/api-rc/niche'
API_NICHE_URL = u'http://localhost:8080/niche'
API_TAXA_URL = u'http://localhost:8080/taxa'

In [ ]:
def get_species_data(api_url, name, analysis_res=16):
    """ Regresa un dataframe con la información taxonomica de una especie y su id en la base de datos  """
    query = {
        "qtype":"getEntList",
        "searchStr":name, # Nombre de la especie substring de la especie a buscar
        "nivel":"especievalidabusqueda",
        "source":1,
        "res_celda_sp":"cells_{}km".format(analysis_res),
        "res_celda_snib":"gridid_{}km".format(analysis_res),
        "res_celda_snib_tb":"grid_{}km_aoi".format(analysis_res)
    }
    r = requests.post(api_url+'/especie', params=query)

    response = json.loads(r.content)
    return pd.DataFrame(response['data'])

In [ ]:
def get_taxon_cells(tax_level, tax_name, grid_res, api_url, fossil=False, sfecha=False):
    """ Regresa un dataframe con las celdas ocupadas por el taxon especificado.
    
    Keyword arguments:
    tax_level -- Nivel taxonomico del taxon, por ejemplo, 'clasevalida' (actualmente está amarrado a los nombres en la base de datos)
    tax_name -- Nombre del taxon, por ejemplo, 'Mammalia'
    grid_res -- La resolución del grid del que queremos obtener las celdas
    api_url -- El URL del endpoint
    """
    query = {
        "tax_level": tax_level,
        "tax_name": tax_name,
        "cells_res": grid_res,
        "sfecha": str(sfecha).lower(),
        "fossil": str(fossil).lower()
    }
    
    r = requests.post(api_url+'/cells', params=query)

    response = json.loads(r.content)
    return pd.DataFrame(response['data'])

In [ ]:
# No está funcionando la especificación de resolución de grid
def get_sp_points(spid, analysis_res=16):
    """ Regresa los registros que se tienen de una especie """
    query = {
        "qtype": "getSpecies",
        "id": spid,
        "sfecha": 'false',
        "fossil": 'false',
        "res_celda_sp":"cells_{}km".format(analysis_res),
        "res_celda_snib":"gridid_{}km".format(analysis_res),
        "res_celda_snib_tb":"grid_{}km_aoi".format(analysis_res)
    }

    r = requests.post(API_NICHE_URL+'/especie', params=query)
    response = json.loads(r.content)
    obs_data = response['data']
    return pd.DataFrame(obs_data)


In [ ]:
def get_biogroup_scores(api_url, spid, analysis_res, group_level, group_name, min_occ = 50, fossil=False, nodate=False):
    """ Para la especie especificada regresa los scores de todas las especies del grupo taxonomico dado """
    query = {
        "qtype":"getGeoRel", 
        "id": spid,
        "res_celda_sp":"cells_{}km".format(analysis_res),
        "res_celda_snib":"gridid_{}km".format(analysis_res),
        "res_celda_snib_tb":"grid_{}km_aoi".format(analysis_res),
        "tfilters[0][field]": group_level,
        "tfilters[0][value]": group_name,
        "tfilters[0][type]": 4,
        "hasBios":"true",
        "hasRaster":"false",
        "sfecha": 'false',
        "fossil": 'false',
        "min_occ": min_occ
    }
    r = requests.post(api_url+'/getGeoRel', params=query)
    response = json.loads(r.content)
    df = pd.DataFrame(response['data'])
    
    
    return df.apply(pd.to_numeric, errors='ignore')

In [ ]:
def get_cells_bioscores(api_url, spid, group_level, group_name, analysis_res, min_occ):
    """ Regresa los scores de todas las celdas usando el grupo taxonómico especificado """
    ftype = 4
    score_name = "{}_score".format(group_name)
    obs_data = True
    rast_data = False
    
    print(spid)
    query = {
            "qtype":"getMapScoreCeldaDecil", 
            "id": spid,
            "idreg":"Estados",
            "idtime": 1506389557454,
            "fossil": False,
            "sfecha": False,
            "apriori": '',
            "discardedDateFilterids": True,
            "val_process": False,
            "idtabla":"no_table",
            "res_celda_sp":"cells_{}km".format(analysis_res),
            "res_celda_snib":"gridid_{}km".format(analysis_res),
            "res_celda_snib_tb":"grid_{}km_aoi".format(analysis_res),
            "tfilters[0][field]": group_level,
            "tfilters[0][value]": group_name,
            "tfilters[0][type]": ftype,
            "hasBios": str(obs_data).lower(),
            "hasRaster": str(rast_data).lower(),
            "min_occ": min_occ
    }

    r = requests.post(API_NICHE_URL+'/getCellScore', params=query)
    response = json.loads(r.content)

    if response.has_key('data'):
        df = pd.DataFrame(response['data'])
        df.rename(columns={'tscore':score_name}, inplace=True)

        return df.apply(pd.to_numeric, errors='ignore')
    

    return pd.DataFrame()

In [ ]:
def get_cells_bioscores_var_n(api_url, spid, group_level, group_name, analysis_res, min_occ):
    """ Regresa los scores de todas las celdas usando el grupo taxonómico especificado """
    ftype = 4
    score_name = "{}_score".format(group_name)
    obs_data = True
    rast_data = False
    
    print(spid)
    query = {
            "qtype":"getMapScoreCeldaDecil", 
            "sp_id": spid,
            "idreg":"Estados",
            "idtime": 1506389557454,
            "fossil": False,
            "sfecha": False,
            "apriori": 'apriori',
            "discardedDateFilterids": True,
            "val_process": False,
            "idtabla":"no_table",
            "cells_res": analysis_res,
            "tfilters[0][field]": group_level,
            "tfilters[0][value]": group_name,
            "tfilters[0][type]": ftype,
            "covar_tax_level": group_level,
            "covar_tax_name": group_name,
            "hasBios": str(obs_data).lower(),
            "hasRaster": str(rast_data).lower(),
            "min_occ": min_occ,
            "n_grid_coverage": 'species_coverage'
    }

    r = requests.post(API_NICHE_URL+'/grid_scores', params=query)
    response = json.loads(r.content)    

    return response

In [ ]:
def get_bio_scores_var_n(api_url, spid, group_level, group_name, analysis_res, min_occ):
    """ Regresa los scores de todas las especies ajustando la n segun la cobertura del grupo taxonómico especificado """

    print(spid)
    query = {
            "sp_id": spid,
            "cells_res": analysis_res,
            "covar_tax_level": group_level,
            "covar_tax_name": group_name,
            "min_occ": min_occ,
            "n_grid_coverage": 'species_coverage'
    }

    r = requests.post(API_NICHE_URL+'/bio_scores', params=query)
    response = json.loads(r.content)    

    return response

In [ ]:
def get_all_species_cell_scores(sps_ids, analysis_res, covar_group_level, covar_group_name, min_occ, api):
    
    while not sps_ids.empty():
        sp_id = sps_ids.pop()
        
        try:
            # get all cells occupied by the current species
            cells_df = get_cells(API_NICHE_URL, sp_id, analysis_res)
            # get all cell scores for the species

            if not cells_df.empty and len(cells_df) >= 5:
                cells_scores = get_cells_bioscores(API_NICHE_URL,
                                                   sp_id, group_level, group_name, analysis_res, min_occ)

                if not cells_scores.empty:
                    count = count + 1
                    print(count)

                    # keep only the cells that the species' registers occupies
                    cells_scores = cells_scores[cells_scores['gridid'].isin(cells_df['cell_id'])]
                    registers_evals[sp_id] = cells_scores
        except ConnectionError as err:
            print(err)
            remaining_ids = set(sps_ids) - set(registers_evals.keys())
            get_all_species_cell_scores(remaining_ids, analysis_res, covar_group_level, covar_group_name, min_occ, api)
    return registers_evals

In [ ]:
def get_data_from_id(api_url, spid):
    r = requests.get("{}/{}".format(API_TAXA_URL, spid))
    response = json.loads(r.content)

    return pd.DataFrame([response])

In [ ]:
# Get all mammal species

def get_children(root_level, root_name, child_level, api_url):
    query = {
        "root_level":root_level,
        "root_name":root_name, # Nombre de la especie substring de la especie a buscar
        "child_level":child_level,
    }
    r = requests.post(api_url+'/children', params=query)

    response = json.loads(r.content)

    return pd.DataFrame(response['data'])

In [ ]:
def get_species_name(sp_id, api_url):
    r = requests.get(api_url+'/{}'.format(sp_id))

    response = json.loads(r.content)

    return response['valid_name']

In [ ]:
def get_cells(api_url, spid, analysis_res, fossil=False, sfecha=False):
    name = get_species_name(spid, API_TAXA_URL)
    q_params = {
        "tax_level": "especievalida",
        "tax_name": name,
        "cells_res": analysis_res,
        "fossil": str(fossil).lower(),
        "sfecha": str(sfecha).lower()
    }
    print(q_params)
    r = requests.post(api_url+'/cells/', params=q_params)
    response = json.loads(r.content)

    df = pd.DataFrame(response['data'])

    return df


# Prueba de método: lo aplicamos a una especie

Analizaremos los registros de *Cynomys mexicanus* (perrito llanero mexicano). 

El proceso de lo que queremos hacer es:
1. Obtener las celdas que ocupa la especie que nos interesa
2. Obtener los scores de todas las celdas del grid para la especie
3. Finlmente nos quedamos solo con los scores de las celdas del punto 1


## Get cells for the selected species

Primero obtenemos el ID de la especie.

In [ ]:
analysis_res = 16

name = "Cynomys mexicanus"
species_data = get_species_data(API_NICHE_URL, name)
species_data

In [ ]:
spid = species_data['spid']

The ID for C. mexicanus is **{{species_data.loc[0,'spid']}}**

In the next query we get all the observations for the species and the cells it occupies

In [ ]:
sp_cells = get_taxon_cells(tax_level="especievalida",
                           tax_name="Cynomys mexicanus",
                           grid_res=analysis_res,
                           api_url=API_NICHE_URL)
sp_cells.cell_id.sort_values()

In [ ]:
obs_df = get_sp_points(spid, analysis_res)

obs_df.gridid.sort_values().unique()

## Get cell scores
En este ejemplo usaremos sólo mamíferos para calcular los scores

In [ ]:
group_level = "clasevalida"
group_name = "Mammalia"
min_occ = 5
species_cells = get_cells_bioscores(API_NICHE_URL,
                                    spid,
                                    group_level,
                                    group_name,
                                    analysis_res,
                                    min_occ)

In [ ]:
species_cells.head()

In [ ]:
species_cells_var_n = get_cells_bioscores_var_n(API_NICHE_URL,
                                    spid,
                                    group_level,
                                    group_name,
                                    analysis_res,
                                    min_occ)

In [ ]:
species_cells_var_n = pd.DataFrame(species_cells_var_n['data'])

In [ ]:
species_cells_var_n.describe()

In [ ]:
species_cells.describe()

## De los resultados seleccionamos solo las celdas que ocupa la especie

In [ ]:
species_cells_var_n = species_cells_var_n[species_cells_var_n.gridid.isin(sp_cells.cell_id)]
species_cells_trad = species_cells[species_cells.gridid.isin(sp_cells.cell_id)]

species_cells_var_n.describe()

In [ ]:
species_cells_trad.describe()

In [ ]:
species_cells_var_n.sort_values(by="tscore", ascending=True).head()

In [ ]:
species_cells = species_cells[species_cells.gridid.isin(sp_cells.cell_id)]
species_cells.sort_values(by="Mammalia_score", ascending=True).head()

## Prueba considerando sólo celdas ocupadas por al menos una especie del grupo de covariables

Obtenemos primero para cada celda la lista de especies, luego hacemos un join con la tabla de scores de especies, finalmente sumamos agrupando por id de celda

In [ ]:
species_scores_var_n = get_bio_scores_var_n(API_NICHE_URL,
                                    spid,
                                    group_level,
                                    group_name,
                                    analysis_res,
                                    min_occ)
species_scores_var_n = pd.DataFrame(species_scores_var_n['data'])

In [ ]:
cells_species = []
cells_scores_dis = []
for c_id in sp_cells.cell_id:
    q_params = {
        "tax_group_level": "clasevalida",
        "tax_group_name": "Mammalia",
        "grid_res": analysis_res,
        "cell_id": c_id
    }
    r = requests.post(API_NICHE_URL+'/cells', params=q_params)
    response = json.loads(r.content)
    cell_sps = response['data']
    [sp.update({'cell_id': c_id}) for sp in cell_sps]
    cells_scores_dis.extend(cell_sps)

cell_sp_df = pd.DataFrame(cells_scores_dis)
cell_scores_dis = pd.merge(species_scores_var_n, cell_sp_df.drop(['var_name'], axis=1), left_on='covar_id',right_on='var_id')
cell_scores = cell_scores_dis[['cell_id', 'score']].groupby('cell_id').sum()

In [ ]:
cell_scores.describe()

En este caso no hubo registros que ocupen una celda con valor negativo

# Aplicar el método para todos los mamíferos



Obtenemos todas las especies de mamífero en la base

In [ ]:
mammals = get_children(root_level="clasevalida",
                       root_name="Mammalia",
                       child_level="especievalidabusqueda",
                       api_url=API_TAXA_URL)

In [ ]:
# Get Mammalia ids 
mammalia_ids = []
for i, mammal in mammals.iterrows():    
    m_data = get_species_data(API_NICHE_URL, mammal['scientific_name'])
    if not m_data.empty:
        mammalia_ids.append(m_data.loc[0,"spid"])

In [ ]:
# Get obs and cells per species
group_level = "reinovalido" 
group_name = "Plantae"

registers_evals = {}

In [ ]:
count = 0
# get cell score for each species register
# remaining_ids = set(mammalia_ids) - set(registers_evals.keys())

In [ ]:
remaining_ids = set(mammalia_ids) - set(registers_evals.keys())

In [ ]:
sp_scores = []
for sp_id in mammalia_ids[:5]:
    new_scores = get_bio_scores_var_n(API_NICHE_URL,
                                        sp_id,
                                        group_level,
                                        group_name,
                                        analysis_res,
                                        min_occ)
    
    
    sp_scores.extend(new_scores['data'])   


In [ ]:
sp_scores

In [ ]:
observations_score = pd.concat(registers_evals)
observ_score_df = pd.DataFrame.from_records(observations_score.to_records())

In [ ]:
observ_score_df

In [ ]:
observ_score_df = observ_score_df[observ_score_df['tscore'] < 0]
observ_score_df.drop('level_1', axis='columns', inplace=True)
observ_score_df.rename(index=str, columns={"level_0": "sp_id"}, inplace=True)
len(observ_score_df)

In [ ]:
sp_ids = observ_score_df.sp_id.unique()
print(len(sp_ids))
ids_map = {}
for sid in sp_ids:
    ids_map[sid] = get_species_name(sid, API_TAXA_URL)
ids_map

In [ ]:
observ_score_df['sp_name'] = observ_score_df.sp_id.map(ids_map)

In [ ]:
# Elimina los cetaceos de la lista, sólo estamos analizando terrestres
cetaceos = get_children("ordenvalido", "Cetacea", "especievalidabusqueda", API_TAXA_URL)

In [ ]:
keep_mammals = set(ids_map.values()) - set(cetaceos['scientific_name'])

In [ ]:
observ_score_df = observ_score_df[observ_score_df.sp_name.isin(keep_mammals)]

In [ ]:
observ_score_df.sort_values(by='tscore', ascending=True)

In [ ]:
sp_stats_scores = {}
for r_key in registers_evals.keys():
    scores_stats = registers_evals[r_key].describe().tscore
    sp_stats_scores[r_key] = scores_stats

In [ ]:
sp_stats_scores_df = pd.DataFrame.from_dict(sp_stats_scores, orient='index')
sp_stats_scores_df

In [ ]:
get_cells(API_NICHE_URL, mammalia_ids[0], analysis_res)

In [ ]:
sp_scores_stats_df[sp_scores_stats_df['count'] > 10].sort_values(by='min', ascending=True)


In [ ]:
sp_data = [get_data_from_id(API_NICHE_URL, id) for id in mammalia_ids]

In [ ]:
sp_data_df = pd.concat(sp_data)
sp_data_df = sp_data_df.set_index(keys='spid')

In [ ]:
sp_scores_stats_df = sp_data_df.merge(sp_stats_scores_df, right_index=True, left_index=True)

In [ ]:
sp_scores_stats_df

## Get the most suspicious observations 

That is, observation registers at cells with the lowest scores

In [ ]:
observ_score_df.sort_values(by='Plantae_score').head()

In [ ]:
observ_score_df[observ_score_df['sp_id'] == 27304]

In [ ]:
selected_species_cells = registers_evals[27754]
print(sp_scores_stats_df.loc[27754])
selected_species_cells

In [ ]:

feature_group_suspicious = folium.FeatureGroup(name='Suspicious')
feature_group_ok = folium.FeatureGroup(name='OK')

# load species points
pt_data = get_sp_points(27754)


In [ ]:
geo_data = [json.loads(e) for e in pt_data['json_geom']]

In [ ]:
len(pt_data)

In [ ]:
points = [pt['coordinates'][::-1] for pt in geo_data]
marker_cluster_active = folium.MarkerCluster()
marker_cluster_unactive =folium.MarkerCluster()

for site in data_sites:
    if(site["status"]=="is_active"):
        marker_active = folium.Marker(site["coordinates"],popup="OK",icon = folium.Icon(color='green',icon='ok-sign'))
        marker_cluster_active.add_child(marker_active)
    else:
        marker_unactive = folium.Marker(site["coordinates"],popup="KO",icon = folium.Icon(color='red',icon='exclamation-sign'))
        marker_cluster_unactive.add_child(marker_unactive)
        
folium.plugins.MarkerCluster(locations=points).add_to(m)

In [ ]:
selected_species_cells['gridid'] = selected_species_cells.index
selected_species_cells

In [ ]:
#
deciles = selected_species_cells.Magnoliopsida_score.quantile([1, .5, ])
deciles = [int(d) for d in deciles]



m = folium.Map(location=[48, -102],
               zoom_start=3
              )
grid_path = os.path.join("../SPECIES/repos/snib-middleware/geofiles/niche/", "mx_grid_{}km.json".format(analysis_res))


m.choropleth(
    grid_path,
    data=selected_species_cells,
    columns=['gridid', 'Magnoliopsida_score'],
    key_on='feature.properties.gridid',
    threshold_scale=[-400, -1, 0, 1, 400],
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_color='white',
    line_opacity=0.2,
    reset=True)

points = [pt['coordinates'][::-1] for pt in geo_data]
folium.plugins.MarkerCluster(locations=points).add_to(m)

m.save('test_validation.html')

In [ ]:
? m.choropleth

In [ ]:
pwd

## Get the most problematic species
That is, species with the largest variance in their occupied cells scores

In [ ]:
get_taxon_cells('clasevalida', 'Mammalia', 16, API_NICHE_URL)

# Por hacer
* Implementar: Especificación de resolución de grid para obtener las celdas que ocupa un taxon a cualquier resolución
* Implementar: Calcular scores sólo para un conjunto de celdas especifico
* Apriori debería ser un booleano en vez de requerir la palabra apriori, ¿o por qué está así?

# Notas
* ¿Tiene sentido considerar la probabilidad de que cualquier especie caiga en una celda dada su riqueza?
    * Ejemplo: Sabemos que una celda C1 tiene registradas 1000 especies y otra celda C2 tiene una. Dada una especie E, sin más información la probabidad de que E esté en C1 es mayor a que esté en C2.


In [ ]:
5 / 10

In [ ]:
def epsilon(n_c, n_x, n_cx, n):
    p_c = n_c / n
    return n_x * (n_cx / n_x - p_c) / (n_x * p_c * (1 - p_c))**0.5

In [ ]:
epsilon(115, 115, 0, 200)

In [ ]:
(1 - 115/20000)**115

In [ ]:
get_

In [ ]:
mamm_cells = get_taxon_cells('clasevalida', 'Mammalia', 16, API_NICHE_URL)

In [ ]:
rep_cells = get_taxon_cells('clasevalida', 'Reptilia', 16, API_NICHE_URL)

In [ ]:
len(set(mamm_cells.cell_id).union(rep_cells.cell_id))

In [ ]:
def epsilon(ni, nj, nij, n):
    p_i = ni / float(n)
    return nj * (nij / float(nj) - p_i) / (nj * p_i * (1 - p_i))**0.5

In [ ]:
epsilon(22, 272, 0, 26944)

In [ ]:
epsilon(22, 272, 0, 8537)

In [ ]:
def score(ni, nj, nij, n, alfa = 0.000001):
    p_i = ni / float(n)
    p_no_i = (n - ni) / float(n)
    
    p_ji = nij / float(ni)
    p_j_no_i = (nj - nij) / float(n - ni)
    
    s_ji = math.log((p_ji + alfa) / (p_j_no_i + 2*alfa))
    
    return s_ji

In [ ]:
score(22, 284, 6, 8537, alfa=0.000001)

Con variables climaticas

In [ ]:
group_ind = 1
group_str = 'tfilters[{}]'.format(group_ind)
group_fields = {'label':'{}[label]', 'type':'{}[type]', 'level':'{}[level]', 'value':'{}[value]'}

for k in group_fields.keys():
    group_fields[k] = group_fields[k].format(group_str)
group_fields    

new_abio_vargroup = {
 group_fields['label']: 'Bioclim',
 group_fields['type']: 0,
 group_fields['level']: 0,
 group_fields['value']: 'root_bioclim',
}

query.update(new_abio_vargroup)

query['hasRaster'] = 'true'
query

score_name = 'magn_abiot_score'
r = requests.post(API_URL+'/getCellScore', params=query)
print(r)
response = json.loads(r.content)

df = pd.DataFrame(response['data'])

df['tscore'] = df['tscore'].apply(pd.to_numeric)
df.rename(columns={'tscore':score_name}, inplace=True)
df.describe()

df.set_index('gridid', inplace=True)
abio_magn_scores = df
abio_magn_scores.describe()

abio_magn_scores = abio_magn_scores.loc[sp_cells.unique()]
abio_magn_scores.describe()

In [ ]:
get_species_name(27336, API_TAXA_URL)